# Testing and Validating for AA2030 Python pipeline

In [1]:
import pandas as pd
import numpy as np
import datetime

Put in the file name here of the constellation data.

In [2]:
#const = pd.read_excel('AAPS_CONST.xlsx')
const = pd.read_excel('Constellation_Excel_input\APPS_CONST_3.31.2022.xlsx')

Put in the file name here of the ESPM upload template.

In [3]:
#up = pd.read_excel('AAPS_Cleaned_Rows1.xlsx', 'Add Bills-Non Electric')
up = pd.read_excel('Output.xlsx', 'Add Bills-Non Electric')

In [4]:
#separating out the constellation meter name from our upload sheet to compare row data from const sheet
up['const_customer_id'] = up['Meter Name\n(Pre-filled)'].str.split('__', expand=True)[1]
up['const_meter_number'] = up['Meter Name\n(Pre-filled)'].str.split('__', expand=True)[2].astype('int64')

In [5]:
len(up['const_meter_number'].unique())

44

In [6]:
len(const['MeterNumber'].unique())
#7 more meters in const then in upload for aaps, due to unfinished school mapping
#52 more meters for the city, due to unfinished mapping and lots of meters are electric

54

In [7]:
#great meters are the meters represented in both the constellation input and in our upload template
great_meters = list(set(const['MeterNumber']) & set(up['const_meter_number']))
len(great_meters)

43

In [8]:
#looking at which meter is in great meters but not in the constellation input
set(up['const_meter_number'].unique()) - set(great_meters)
#I think this meter is getting messed up bc our upload version is missing it's 10th digit! (6)

{248907587}

In [9]:
#now actually iterating through meter data to validate that info is lining up
for meter in great_meters:
    #grabbing relevant rows from each spreadsheet that have the same meter number
    #grabbing last occurence to get most recent bill data
    const_row = const.loc[const['MeterNumber'] == meter].iloc[-1]
    #grabbing 2nd occurence to get most recent billing data
    up_row = up.loc[up['const_meter_number'] == meter].iloc[1]
    
    #asserting that the row values are the same between the sheets
    assert const_row['CustomerId'] == up_row['const_customer_id'], f"Customer ID mismatch, {meter}"
    #asserting that most recent bill data has correct cycle start date
    print(const_row['CycleStartDate'], up_row['Start Date\n(Required)'])
    assert const_row['CycleStartDate'] == up_row['Start Date\n(Required)'], f"Start Date mismatch, {meter}"
    #asserting that most recent bill data has correct cycle end date
    assert const_row['CycleEndDate'] == up_row['End Date\n(Required)'], f"End Date mismatch, {meter}"
    #asserting that most recent bill data has correct quantity
    assert const_row['FeeVolume'] == up_row['Quantity\n(Required)'], f"Quantity mismatch, {meter}"
    #checking if estimation value is correct 
    if const_row['EndReadType'] == 'Actual':
        assert up_row['Estimation (Required)'] == 'No'
    if const_row['EndReadType'] == 'Estimate':
        assert up_row['Estimation (Required)'] == 'Yes'    
    #asserting that most recent bill data has correct cost, need to manually go over NaNs
    if np.isnan(const_row['TotalCharges']) & np.isnan(up_row['Cost\n(Optional)']):
        continue
        assert const_row['TotalCharges'] == up_row['Cost\n(Optional)'], f"Cost mismatch, {meter}"
        

2022-02-08 00:00:00 2022-02-08 00:00:00
2022-01-26 00:00:00 2022-01-26 00:00:00
2022-01-22 00:00:00 2022-01-22 00:00:00
2022-01-21 00:00:00 2022-01-21 00:00:00
2021-10-19 00:00:00 2021-10-19 00:00:00
2022-01-06 00:00:00 2022-01-06 00:00:00
2022-01-19 00:00:00 2022-01-19 00:00:00
2022-01-11 00:00:00 2022-01-11 00:00:00
2022-01-04 00:00:00 2022-01-04 00:00:00
2022-01-19 00:00:00 2022-01-19 00:00:00
2022-01-08 00:00:00 2022-01-08 00:00:00
2022-01-06 00:00:00 2022-01-06 00:00:00
2022-02-01 00:00:00 2022-02-01 00:00:00
2022-01-22 00:00:00 2022-01-22 00:00:00
2022-01-25 00:00:00 2022-01-25 00:00:00
2022-01-29 00:00:00 2022-01-29 00:00:00
2022-01-22 00:00:00 2022-01-22 00:00:00
2022-01-24 00:00:00 2022-01-24 00:00:00
2022-01-15 00:00:00 2022-01-15 00:00:00
2022-01-05 00:00:00 2022-01-05 00:00:00
2022-01-22 00:00:00 2022-01-22 00:00:00
2022-01-13 00:00:00 2022-01-13 00:00:00
2022-01-14 00:00:00 2022-01-14 00:00:00
2022-01-21 00:00:00 2022-01-21 00:00:00
2020-10-22 00:00:00 2020-10-22 00:00:00


## Part 2: Sanity Checks 

In [10]:
up.head(2)

,Meter ID\n(Pre-filled),Meter Consumption ID\n(Pre-filled),Portfolio Manager ID\n(Pre-filled),Property Name\n(Pre-filled),Meter Name\n(Pre-filled),Meter Type\n(Pre-filled),Start Date\n(Required),End Date\n(Required),Quantity\n(Required),Meter Unit\n(Pre-filled),Cost\n(Optional),Estimation (Required),const_customer_id,const_meter_number
0,129002335,NaN,5227,King Elementary,Constellation__RG-43035783__2428262223,Natural Gas,NaT,NaT,NaN,ccf (hundred cubic feet),NaN,NaN,RG-43035783,2428262223
1,129002335,NaN,5227,King Elementary,Constellation__RG-43035783__2428262223,Natural Gas,2022-01-26,2022-02-24,8530.0,ccf (hundred cubic feet),4697.51,No,RG-43035783,2428262223


In [11]:
const.head(2)

,Entity,CustomerId,MeterNumber,street,ProdMonth,EndReadType,CycleStartDate,CycleEndDate,BeginningReading,EndReading,MeterMultiplier,FeeVolume,TotalCharges
0,Ann Arbor Public Schools,RG-43035759,2110022691,2555 S STATE ST,2020-11-01,Actual,2020-10-24,2020-11-24,922156,938752,1.1195,18139.0,9679.92
1,Ann Arbor Public Schools,RG-43035760,4031211372,2555 S STATE ST,2020-11-01,Actual,2020-10-10,2020-11-09,1573,1714,1.0000,141.0,75.26


In [12]:
up['Start Date\n(Required)'].value_counts().sort_index()

2020-10-02    2
2020-10-03    1
2020-10-06    2
2020-10-07    2
2020-10-09    3
             ..
2022-01-29    1
2022-02-01    1
2022-02-02    1
2022-02-04    1
2022-02-08    1
Name: Start Date\n(Required), Length: 267, dtype: int64

In [13]:
up['End Date\n(Required)'].value_counts().sort_index()
#just looking at these to make sure the time range makes sense

2020-11-02    1
2020-11-03    2
2020-11-04    3
2020-11-05    1
2020-11-06    2
             ..
2022-03-01    1
2022-03-02    1
2022-03-03    1
2022-03-04    1
2022-03-07    1
Name: End Date\n(Required), Length: 272, dtype: int64

We can see from the "date" value counts here that there aren't any extreme outliers in bill dates, all being within the past year or two so there don't seem to be any major typos here. 

In [14]:
#want to verify that manually entered ESPM meter names are in the correct format
import re

for meter in up['Meter Name\n(Pre-filled)']:
    if meter == 'Constellation__RG-43035789__248907587':
        continue
    assert re.match('Constellation__RG-[0-9]{6,8}__[0-9]{10}', meter), f"{meter} is formatted wrong"

In [15]:
up['Quantity\n(Required)'].value_counts().sort_index()
#verifying quantities aren't bizarre

0.00        51
2.00         1
3.00         1
5.00         2
9.00         2
            ..
28689.01     1
32556.02     1
36776.00     1
39571.00     1
39874.00     1
Name: Quantity\n(Required), Length: 532, dtype: int64